In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("all_kindle_review.csv")
df = df[['reviewText', 'rating']].copy()
df

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4
...,...,...
11995,Valentine cupid is a vampire- Jena and Ian ano...,4
11996,I have read all seven books in this series. Ap...,5
11997,This book really just wasn't my cuppa. The si...,3
11998,"tried to use it to charge my kindle, it didn't...",1


In [2]:
df.shape

(12000, 2)

In [3]:
df.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [4]:
df['rating'].unique()

array([3, 5, 4, 2, 1], dtype=int64)

In [5]:
df['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

In [6]:
## Preprocessing and Cleaning
df['rating'] = df['rating'].apply(lambda x:0 if x<3 else 1) #positive=1, negative=0
df

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1
...,...,...
11995,Valentine cupid is a vampire- Jena and Ian ano...,1
11996,I have read all seven books in this series. Ap...,1
11997,This book really just wasn't my cuppa. The si...,1
11998,"tried to use it to charge my kindle, it didn't...",0


In [7]:
df['rating'].value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

In [8]:
#lowercase all text
df['reviewText']=df['reviewText'].str.lower()

In [9]:
import re
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
nltk.download('stopwords')
# removing special characters
df['reviewText'] = df['reviewText'].apply(lambda x: re.sub('[^a-z A-Z 0-9-]+', '', x))
# removing stopwords
df['reviewText'] = df['reviewText'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords.words('english')]))
# removing html url
df['reviewText'] = df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-])?', '', str(x)))
# removing html tags
df['reviewText']=df['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())
# removing any extra spaces
df['reviewText'] = df['reviewText'].apply(lambda x: " ".join(x.split()))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\soumasnigdha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
df['reviewText'].head(10)

0    jace rankin may short hes nothing mess man hau...
1    great short read didnt want put read one sitti...
2    ill start saying first four books wasnt expect...
3    aggie angela lansbury carries pocketbooks inst...
4    expect type book library pleased find price right
5    aislinn little girl big dreams death older bro...
6    makings good story unfortunately disappointsit...
7    got like collaborated short stories alot times...
8    loved book hooked series hope kelsey maxwell r...
9    thats good thing short sweet tease gives every...
Name: reviewText, dtype: object

In [11]:
## lemmetization
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
df['reviewText'] = df['reviewText'].apply(lambda x: lemmatize_words(x))

In [12]:
df.head(10)

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1
5,aislinn little girl big dream death older brot...,1
6,making good story unfortunately disappointsit ...,0
7,got like collaborated short story alot time tw...,1
8,loved book hooked series hope kelsey maxwell r...,1
9,thats good thing short sweet tease give everyt...,1


In [13]:
# train test split
from sklearn.model_selection import train_test_split
X = df['reviewText']
y = df['rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((9600,), (2400,), (9600,), (2400,))

In [14]:
# bag of words
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer()
X_train_bow = bow.fit_transform(X_train).toarray()
X_test_bow = bow.transform(X_test).toarray()

In [15]:
# tfidf
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()

In [16]:
df['reviewText'].head(10)

0    jace rankin may short he nothing mess man haul...
1    great short read didnt want put read one sitti...
2    ill start saying first four book wasnt expecti...
3    aggie angela lansbury carry pocketbook instead...
4    expect type book library pleased find price right
5    aislinn little girl big dream death older brot...
6    making good story unfortunately disappointsit ...
7    got like collaborated short story alot time tw...
8    loved book hooked series hope kelsey maxwell r...
9    thats good thing short sweet tease give everyt...
Name: reviewText, dtype: object

In [90]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

words = []
for sent in X_train:
  sent_token = sent_tokenize(sent)
  for sent in sent_token:
    words.append(simple_preprocess(sent))

X_train_avgword2vec = words
X_train_avgword2vec

[['looking',
  'forward',
  'book',
  'came',
  'double',
  'space',
  'every',
  'paragraph',
  'kindle',
  'edition',
  'since',
  'action',
  'move',
  'around',
  'formatting',
  'make',
  'story',
  'hard',
  'follow',
  'die',
  'hard',
  'like',
  'want',
  'botherits',
  'sad',
  'thing',
  'good',
  'book',
  'spoiled',
  'formatting',
  'fault',
  'author',
  'story',
  'good',
  'book',
  'energy',
  'read',
  'itive',
  'also',
  'emailed',
  'author'],
 ['already',
  'owned',
  'book',
  'spouse',
  'forgot',
  'already',
  'part',
  'library',
  'book',
  'unfortunate',
  'amazon',
  'safeguard',
  'place',
  'notify',
  'book',
  'already',
  'library'],
 ['cool',
  'forgot',
  'request',
  'rate',
  'came',
  'make',
  'mine',
  'unreliable',
  'rating',
  'doubt',
  'change',
  'rotation',
  'planet',
  'enough',
  'measure'],
 ['short',
  'short',
  'story',
  'basically',
  'scene',
  'party',
  'one',
  'night',
  'though',
  'admittedly',
  'entire',
  'book',
  't

In [85]:
words = []
for sent in X_test:
  sent_token = sent_tokenize(sent)
  for sent in sent_token:
    words.append(simple_preprocess(sent))

X_test_avgword2vec = words
X_test_avgword2vec

[['really', 'great', 'read', 'wish', 'would', 'hope', 'find', 'author'],
 ['nope',
  'tried',
  'cant',
  'read',
  'take',
  'greatest',
  'delight',
  'deleting',
  'kindle',
  'total',
  'waste',
  'time'],
 ['story',
  'line',
  'drug',
  'like',
  'book',
  'much',
  'mystery',
  'fan',
  'would',
  'tell',
  'worth',
  'reading'],
 ['read',
  'several',
  'angel',
  'book',
  'one',
  'work',
  'didnt',
  'really',
  'feel',
  'much',
  'story',
  'line',
  'didnt',
  'fall',
  'love',
  'character',
  'postive',
  'note',
  'author',
  'good',
  'writing',
  'sex',
  'scene',
  'tired',
  'reading',
  'second',
  'one',
  'skipping',
  'past',
  'get',
  'back',
  'true',
  'story',
  'find',
  'conflict',
  'going',
  'feel',
  'book',
  'sexual',
  'chemistry',
  'dark',
  'angel',
  'human',
  'conflict'],
 ['possibly',
  'worst',
  'book',
  'ever',
  'read',
  'beginning',
  'positively',
  'excrable',
  'see',
  'book',
  'started',
  'author',
  'young',
  'sex',
  'scenc

In [91]:
# average word2vec
import gensim
from gensim.models import Word2Vec, KeyedVectors
import gensim.downloader as api

wv = api.load("word2vec-google-news-300")

def get_average_word2vec(sentence):
    return np.mean([wv[word] for word in sentence if word in wv.key_to_index], axis=0)

all_sentence_vectors_train = []

for sentence in X_train_avgword2vec:
    sentence_vector = get_average_word2vec(sentence)
    all_sentence_vectors_train.append(sentence_vector)

X_train_avgword2vec = np.array(all_sentence_vectors_train)


all_sentence_vectors_test = []

for sentence in X_test_avgword2vec:
    sentence_vector = get_average_word2vec(sentence)
    all_sentence_vectors_test.append(sentence_vector)

X_test_avgword2vec = np.array(all_sentence_vectors_test)


In [99]:
X_train_avgword2vec

array([[ 0.05390854,  0.02561035,  0.00954399, ..., -0.01777115,
        -0.00395761,  0.00055313],
       [-0.00756028, -0.00208552,  0.0045184 , ..., -0.00425092,
        -0.00692929, -0.0473184 ],
       [ 0.01639404,  0.07115886,  0.06859538, ..., -0.08129476,
         0.02947998,  0.01456833],
       ...,
       [ 0.05996839,  0.0052405 ,  0.02284566, ..., -0.05392456,
        -0.00291561,  0.02201002],
       [ 0.06612549,  0.0540081 ,  0.00936356, ..., -0.04873962,
         0.03250313, -0.02253189],
       [ 0.01938361,  0.06315023,  0.01314249, ..., -0.03160203,
         0.01851573, -0.02172298]], dtype=float32)

In [100]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

nb_model_bow = GaussianNB()
nb_model_bow.fit(X_train_bow, y_train)
y_pred_bow = nb_model_bow.predict(X_test_bow)
print("Naive Bayes with Bag of Words")
print("Accuracy:", accuracy_score(y_test, y_pred_bow))
print("Classification Report:\n", classification_report(y_test, y_pred_bow))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_bow))

print("*"*50)

nb_model_tfidf = GaussianNB()
nb_model_tfidf.fit(X_train_tfidf, y_train)
y_pred_tfidf = nb_model_tfidf.predict(X_test_tfidf)
print("Naive Bayes with TF-IDF")
print("Accuracy:", accuracy_score(y_test, y_pred_tfidf))
print("Classification Report:\n", classification_report(y_test, y_pred_tfidf))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_tfidf))

print("*"*50)

rf_model_avgword2vec = RandomForestClassifier(random_state=42)
rf_model_avgword2vec.fit(list(X_train_avgword2vec), y_train)
y_pred_avgword2vec = rf_model_avgword2vec.predict(list(X_test_avgword2vec))
print("Random Forest with Average Word2Vec")
print("Accuracy:", accuracy_score(y_test, y_pred_avgword2vec))
print("Classification Report:\n", classification_report(y_test, y_pred_avgword2vec))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_avgword2vec))

print("*"*50)


Naive Bayes with Bag of Words
Accuracy: 0.5745833333333333
Classification Report:
               precision    recall  f1-score   support

           0       0.41      0.62      0.49       803
           1       0.74      0.55      0.63      1597

    accuracy                           0.57      2400
   macro avg       0.58      0.59      0.56      2400
weighted avg       0.63      0.57      0.59      2400

Confusion Matrix:
 [[499 304]
 [717 880]]
**************************************************
Naive Bayes with TF-IDF
Accuracy: 0.57875
Classification Report:
               precision    recall  f1-score   support

           0       0.41      0.61      0.49       803
           1       0.74      0.56      0.64      1597

    accuracy                           0.58      2400
   macro avg       0.58      0.59      0.57      2400
weighted avg       0.63      0.58      0.59      2400

Confusion Matrix:
 [[488 315]
 [696 901]]
**************************************************
Random Fore